2.2 — Code a mini-BPE learner

Toy Corpus: low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new



1.	Use the classroom code above (or your own) to learn BPE merges for the toy corpus.
o	Print the top pair at each step and the evolving vocabulary size.


In [17]:
from collections import Counter, defaultdict

# 1. Original corpus with frequencies
corpus_tokens = """
low low low low low lowest lowest newer newer newer
newer newer newer wider wider wider new new
""".split()

def make_vocab(tokens):
    """
    Building initial vocab: chars + </w>(end of word) with frequencies
    """
    freqs = Counter(tokens)
    vocab = Counter()
    for w, f in freqs.items():
        # appending space ' '  between the characters and also end of work marker </w>
        chars = " ".join(list(w)) + " </w>"
        vocab[chars] += f

    print('vocab : ',vocab)
    return vocab

vocab = make_vocab(corpus_tokens)

def get_frequencies(vocab):
    """
    Counting adjacent character pair frequencies
    """
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        # counting adj pair frequencies
        for i in range(len(symbols)-1):
            pairs[(symbols[i], symbols[i+1])] += freq
    return pairs

def merge_vocab(pair, vocab):
    """
    Merging pairs of characters in vocab
    """
    a, b = pair
    bigram = " ".join(pair)
    merged = a + b
    new_vocab = Counter()
    for word, freq in vocab.items():
        new_word = word.replace(bigram, merged)
        new_vocab[new_word] += freq
    return new_vocab

# 2. BPE(Byte-Pair Encoding) merging
print("Initial vocab size:", len(vocab))
merges = []

# no.of steps for merging. setting default to 15
no_of_merge_steps = 15

for i in range(no_of_merge_steps):
    stats = get_frequencies(vocab)
    if not stats:
        break
    # getting the most frequent adj pair based on the frequency count
    best = max(stats, key=stats.get)
    merges.append(best)
    print('merges : ', merges)

    print(f"Step {i+1}: '{best[0]}+{best[1]}' (count={stats[best]})")
    print(f"Vocab size: {len(vocab)} → {len(merge_vocab(best, vocab))}")
    vocab = merge_vocab(best, vocab)


vocab :  Counter({'n e w e r </w>': 6, 'l o w </w>': 5, 'w i d e r </w>': 3, 'l o w e s t </w>': 2, 'n e w </w>': 2})
Initial vocab size: 5
merges :  [('e', 'r')]
Step 1: 'e+r' (count=9)
Vocab size: 5 → 5
merges :  [('e', 'r'), ('er', '</w>')]
Step 2: 'er+</w>' (count=9)
Vocab size: 5 → 5
merges :  [('e', 'r'), ('er', '</w>'), ('n', 'e')]
Step 3: 'n+e' (count=8)
Vocab size: 5 → 5
merges :  [('e', 'r'), ('er', '</w>'), ('n', 'e'), ('ne', 'w')]
Step 4: 'ne+w' (count=8)
Vocab size: 5 → 5
merges :  [('e', 'r'), ('er', '</w>'), ('n', 'e'), ('ne', 'w'), ('l', 'o')]
Step 5: 'l+o' (count=7)
Vocab size: 5 → 5
merges :  [('e', 'r'), ('er', '</w>'), ('n', 'e'), ('ne', 'w'), ('l', 'o'), ('lo', 'w')]
Step 6: 'lo+w' (count=7)
Vocab size: 5 → 5
merges :  [('e', 'r'), ('er', '</w>'), ('n', 'e'), ('ne', 'w'), ('l', 'o'), ('lo', 'w'), ('new', 'er</w>')]
Step 7: 'new+er</w>' (count=6)
Vocab size: 5 → 5
merges :  [('e', 'r'), ('er', '</w>'), ('n', 'e'), ('ne', 'w'), ('l', 'o'), ('lo', 'w'), ('new', 'er</w

2.	Segment the words: new, newer, lowest, widest, and one word you invent (e.g., newestest).
o	Include the subword sequence produced (tokens with _ where applicable).


In [3]:
# Segmenting test words (with _ convention)
def get_tokens(word, merges):
    """
    Applying merges to get subword tokens
    """
    symbols = list(word) + ["</w>"]
    print(f'word : {word} --> symbols : {symbols}')
    for a, b in merges:
        i = 0
        # check if the merge characters are present in the current word symbols and merge
        while i < len(symbols)-1:
            if symbols[i] == a and symbols[i+1] == b:
                symbols[i] = a + b
                del symbols[i+1]
            else:
                i += 1
    if symbols[-1] == "</w>":
        symbols.pop()
    # Convert to subword format with _
    tokens = []
    for s in symbols:
        if s == ' ':
            tokens[-1] += '_'
        else:
            tokens.append(s.replace(' ', '_'))
    return tokens

test_words = ["new", "newer", "lowest", "widest", "newestest"]
print("\nSegmentations:")
for w in test_words:
    tokens = get_tokens(w, merges)
    print(f"{w} → {' '.join(tokens)}")


Segmentations:
word : new --> symbols : ['n', 'e', 'w', '</w>']
new → new
word : newer --> symbols : ['n', 'e', 'w', 'e', 'r', '</w>']
newer → newer</w>
word : lowest --> symbols : ['l', 'o', 'w', 'e', 's', 't', '</w>']
lowest → lowest</w>
word : widest --> symbols : ['w', 'i', 'd', 'e', 's', 't', '</w>']
widest → wid e s t
word : newestest --> symbols : ['n', 'e', 'w', 'e', 's', 't', 'e', 's', 't', '</w>']
newestest → new e s t e s t


3.	In 5–6 sentences, explain:
o	How subword tokens solved the OOV (out-of-vocabulary) problem.
o	One example where subwords align with a meaningful morpheme (e.g., er_ as English agent/comparative suffix).

BPE Explanation
Subword tokenization solves the out-of-vocabulary (OOV) problem by breaking unknown words into smaller pieces drawn from a fixed vocabulary of character sequences. Instead of mapping rare words like "newestest" to an <UNK> token, BPE composes it from learned subwords (new, est, est) that already exist in the vocabulary. This makes the token sequence interpretable and allows models to generalize across morphological patterns.

One meaningful alignment occurs with er_ and est_ suffixes, which emerge naturally from the corpus frequency. In "newer→new er_" and "lowest→low est_", these subwords correspond to English comparative/superlative morphemes, letting the model capture productive grammatical patterns without explicit linguistic rules.

2.3 — Your language (or English if you prefer)
Pick one short paragraph (4–6 sentences) in your own language (or English if that’s simpler).




1.	Train BPE on that paragraph (or a small file of your choice).
o	Use end-of-word _.
o	Learn at least 30 merges (adjust if the text is very small).

In [43]:
from collections import Counter, defaultdict
import re

# Training paragraph (English)
paragraph = """
At 186 miles per hour, the landscape starts to blur. A mile disappears every 20 seconds. An entire town can blink by in the time it takes to remember its name. High-speed trains are, as the name suggests, fast. When two are traveling toward each other, the distance between them can close at double their top speed. Sixty seconds after they pass — a moment when the drivers barely have the chance to exchange a friendly wave — they can be six miles apart, relentlessly powering across the countryside, each with more than 1,000 passengers on board. That breathtaking pace makes the recent deadly crash between two high-speed trains in Spain even more alarming. Despite an extraordinary global safety record, the derailment on a straight stretch of track was a stark reminder of how exceptional, and how fragile, these systems can be.
"""

# Using end-of-word _ instead of </w>
def preprocess(corpus):
    """Splitting into words, adding _ end marker"""
    words = re.findall(r'\b\w+\b', corpus.lower())
    vocab = Counter()
    for w in words:
        chars = " ".join(list(w)) + " _"
        vocab[chars] += 1
    return vocab

vocab = preprocess(paragraph)
print(f"Initial vocab size: {len(vocab)}")

def get_stats(vocab):
    pairs = Counter()
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[(symbols[i], symbols[i+1])] += freq
    return pairs

def merge_vocab(pair, vocab):
    a, b = pair
    bigram = f"{a} {b}"
    merged = a + b
    new_vocab = Counter()
    for word, freq in vocab.items():
        new_word = word.replace(bigram, merged)
        new_vocab[new_word] += freq
    return new_vocab

# Training 30+ merges
merges = []
merge_count = 35
for i in range(merge_count):
    stats = get_stats(vocab)
    if not stats:
        break
    best = max(stats, key=stats.get)
    merges.append(best)
    vocab = merge_vocab(best, vocab)

print(f"\nLearned {len(merges)} merges")

Initial vocab size: 105

Learned 35 merges


2.	Show the five most frequent merges and the resulting five longest subword tokens.


In [45]:
# Showing 5 most frequent merges & 5 longest subword tokens
print("\nTop 5 most frequent merges:")
for i, (a,b) in enumerate(merges[:5], 1):
    print(f"{i}: '{a}+{b}'")

print("\n5 longest subword tokens in final vocab:")
longest = sorted(vocab, key=len, reverse=True)[:5]
for token in longest:
    print(f"  {repr(token)} (freq: {vocab[token]})")


Top 5 most frequent merges:
1: 'e+_'
2: 's+_'
3: 't+h'
4: 'n+_'
5: 'a+r'

5 longest subword tokens in final vocab:
  'e x c e p t i o n a l _' (freq: 1)
  'e x tra o r d in ar y_' (freq: 1)
  're l en t l e s s ly_' (freq: 1)
  'co u n tr y s i d e_' (freq: 1)
  'b re a th t a k ing_' (freq: 1)


3.	Segment 5 different words from the paragraph:
o	Include one rare word and one derived/inflected form.


In [46]:
# Segmentation
def segment(word, merges):
    symbols = list(word.lower()) + ["_"]
    for a, b in merges:
        i = 0
        while i < len(symbols)-1:
            if symbols[i] == a and symbols[i+1] == b:
                symbols[i] = a + b
                del symbols[i+1]
            else:
                i += 1
    if symbols[-1] == "_":
        symbols.pop()
    return symbols

# 5 words - including one rare and one derived/inflected form
test_words = ["landscape", "stark", "blur", "alarming", "1,000"]
print("\nSegmentations:")
for w in test_words:
    tokens = segment(w, merges)
    print(f"{w} → {' '.join(tokens)}")


Segmentations:
landscape → l an d s c a p e_
stark → st ar k
blur → b l u r
alarming → a l ar m ing_
1,000 → 1 , 0 0 0


4.	Brief reflection (5–8 sentences):
o	What kinds of subwords were learned (prefixes, suffixes, stems, whole words)?
o	Two concrete pros/cons of subword tokenization for your language.

Reflection:

With 35 merges, BPE learned mostly s+_, e+_ suffixes (morphology) and t+h, a+r bigrams (inside-word patterns). No prefixes or stems emerged due to small corpus size. Longest "subwords" were actually whole rare words that didn't get merged.

PRO 1: Shared suffixes (s+_) mean "trains_" and "starts_" use same ending leading to morphological generalization.
PRO 2: Handles inflection automatically. Meaning any new plural gets s+_.
CON 1: Rare words like "exceptional_" stay character-level (11 tokens) - verbose encoding.
CON 2: Small corpus limited us to 5 useful merges; needs larger text corpus for real morphology.



Q5. Programming Question:

2. Compare with a Tool
Run the paragraph through an NLP tool that supports your language (e.g., NLTK, spaCy, or any open-source tokenizer if available).
•	Compare tool output vs. your manual tokens.
•	Which tokens differ? Why?


Paragraph : A stroll along Mohegan Bluffs on Block Island is one of the most dramatic walks available in the Ocean State. The bluffs stand 200 feet above the beach and, on a clear day, provide views straight across the Block Island Sound to Montauk, New York. After a stroll, check out the picturesque lighthouses that dot Block Island.

In [41]:
import re
import nltk

paragraph = """A stroll along Mohegan Bluffs on Block Island is one of the most dramatic walks available in the Ocean State. The bluffs stand 200 feet above the beach and, on a clear day, provide views straight across the Block Island Sound to Montauk, New York. After a stroll, check out the picturesque lighthouses that dot Block Island."""

def manual_tokenize(text):
    tokens = text.split()
    result = []
    for token in tokens:
        if token.endswith('.'):
            result.extend([token[:-1], '.'])
        elif token.endswith(','):
            result.extend([token[:-1], ','])
        else:
            result.append(token)
    return result

naive_tokens = paragraph.split()
manual_tokens = manual_tokenize(paragraph)
nltk_tokens = nltk.word_tokenize(paragraph)

print(f'\nNaive Tokens : {naive_tokens}')
print(f'\nManual Tokens : {manual_tokens}')
print(f'\nNLTK Tokens : {nltk_tokens}')

print("-" * 100)
print("TOKEN COMPARISON :")

# comparing naive and manual tokens using prefix matching


manual_idx = 0 # tracking current manual token position

print("\nComparison Results (Naive vs Manual):")
print(f"{'Naive Pos':<8} {'Old Token':<12} {'Manual Pos':<10} {'New Tokens':<20} {'Match?'}")
print("-" * 70)

for naive_idx, old_token in enumerate(naive_tokens):
    if old_token.endswith(('.', ',')):
        # Check if manual tokens match the prefix
        expected_prefix = old_token[:-1]

        # Look ahead in manual_tokens for prefix match
        if (manual_idx < len(manual_tokens) and
            manual_tokens[manual_idx] == expected_prefix and
            manual_idx + 1 < len(manual_tokens) and
            manual_tokens[manual_idx + 1] == old_token[-1]):

            print(f"{naive_idx:<8} '{old_token}'{'':<4} {manual_idx}:{manual_idx+1}{'':<4} "
                  f"'{manual_tokens[manual_idx]}' + '{manual_tokens[manual_idx+1]}'      YES")
            manual_idx += 2  # Skip both word and punctuation
        else:
            print(f"{naive_idx:<8} '{old_token}'{'':<4} {manual_idx}{'':<10} "
                  f"MISMATCH! Expected '{expected_prefix}'      NO")
    else:
        # Non-punctuation token - should match exactly
        if manual_idx < len(manual_tokens) and manual_tokens[manual_idx] == old_token:
            manual_idx += 1
        else:
            print(f"{naive_idx:<8} '{old_token}' → MISMATCH at manual[{manual_idx}]     ")


print(f"\nSummary: Processed {len(naive_tokens)} naïve -> {len(manual_tokens)} manual tokens")
print(f"Manual pointer ended at: {manual_idx} (should be {len(manual_tokens)})")

print("-" * 100)
print("\nComparison Results (Manual vs NLTK):")

# comparing manual and nltk tokens
if(len(manual_tokens) == len(nltk_tokens)):
    mismatch_count = 0
    for manual_idx, manual_token in enumerate(manual_tokens):
        if manual_token != nltk_tokens[manual_idx]:
            print(f'Manual Token : {manual_token} does not match with NLTK Token : {nltk_tokens[manual_idx]} at Index : {manual_idx}')
            mismatch_count += 1

    if mismatch_count == 0:
        print(f'All {len(manual_tokens)} Manual Tokens match NLTK Tokens')
    else:
        print(f' {mismatch_count} Manual Tokens did not match NLTK Tokens')

else:
    print("Length of manual tokens does not match length of NLTK tokens")




Naive Tokens : ['A', 'stroll', 'along', 'Mohegan', 'Bluffs', 'on', 'Block', 'Island', 'is', 'one', 'of', 'the', 'most', 'dramatic', 'walks', 'available', 'in', 'the', 'Ocean', 'State.', 'The', 'bluffs', 'stand', '200', 'feet', 'above', 'the', 'beach', 'and,', 'on', 'a', 'clear', 'day,', 'provide', 'views', 'straight', 'across', 'the', 'Block', 'Island', 'Sound', 'to', 'Montauk,', 'New', 'York.', 'After', 'a', 'stroll,', 'check', 'out', 'the', 'picturesque', 'lighthouses', 'that', 'dot', 'Block', 'Island.']

Manual Tokens : ['A', 'stroll', 'along', 'Mohegan', 'Bluffs', 'on', 'Block', 'Island', 'is', 'one', 'of', 'the', 'most', 'dramatic', 'walks', 'available', 'in', 'the', 'Ocean', 'State', '.', 'The', 'bluffs', 'stand', '200', 'feet', 'above', 'the', 'beach', 'and', ',', 'on', 'a', 'clear', 'day', ',', 'provide', 'views', 'straight', 'across', 'the', 'Block', 'Island', 'Sound', 'to', 'Montauk', ',', 'New', 'York', '.', 'After', 'a', 'stroll', ',', 'check', 'out', 'the', 'picturesque',

2. Compare with a Tool
Run the paragraph through an NLP tool that supports your language (e.g., NLTK, spaCy, or any open-source tokenizer if available).
•	Compare tool output vs. your manual tokens.
•	Which tokens differ? Why?


Tokenized the paragraph using NLTK. Below is the summary:
• Naïve:     57 tokens
• Manual:    64 tokens
• NLTK:      64 tokens

Comparison Results(Manual vs NLTK):

All 64 Manual tokens match NLTK tokens

Comparison Results(Naive vs Manual):

Naive Pos Old Token    Manual Pos New Tokens           Match?
----------------------------------------------------------------------
19       'State.'     19:20     'State' + '.'      YES
28       'and,'     29:30     'and' + ','      YES
32       'day,'     34:35     'day' + ','      YES
42       'Montauk,'     45:46     'Montauk' + ','      YES
44       'York.'     48:49     'York' + '.'      YES
47       'stroll,'     52:53     'stroll' + ','      YES
56       'Island.'     62:63     'Island' + '.'      YES

Summary: Processed 57 naïve → 64 manual tokens
Manual pointer ended at: 64 (should be 64)





3. Multiword Expressions (MWEs)
Identify at least 3 multiword expressions (MWEs) in your language. Example:
•	Place names, idioms, or common fixed phrases.
•	Explain why they should be treated as single tokens.


1. Mohegan Bluffs - refers to a particular landmark on Block Island, not generic “Mohegan” + “bluffs.” As a single token, this represents one landmark entity
2. Block Island - This is a specific geographic location, not just any “block” and any “island.”
3. Ocean State - This is official nickname of Rhode Island
4. Block Island Sound - Block Island Sound is a roughly 10mile wide, 40meter deep marine sound in the Atlantic Ocean separating Block Island from mainland Rhode Island.

The above MWEs should be treated as single tokens because, their meaning is not just the sum of individual words. They correspond to a single named entity/ land mark / location

4. Reflection (5–6 sentences)

Yes, punctuation, morphology, and MWEs significantly complicate tokenization.

1. Punctuation attaches to words without spaces. Tracking index shifts when splitting punctuation from words during the manual tokenization was time consuming however the prefix-matching comparison perfectly validates manual tokenization.
2. English tokenizers rarely split morphemes, but stemming/lemmatization needs morphology
3. MWE need external knowledge/lexicon to tokenize them

Libraries like NLTK, spaCy automatically solve most of these issues and make it look easy but manually tokenizing the paragraph revealed the level of difficulty and the external knowledge/lexicon that is needed to tokenize.

